In [ ]:
!pip install qiskit qiskit-ibm-runtime pylatexenc qiskit_aer

In [ ]:

import qiskit
import numpy as np
qiskit.version.get_version_info()


In [ ]:
from qiskit import QuantumCircuit, QuantumRegister
# 3-qubit simon oracle with b = 101 을 정의하는 함수
def simon_oracle3(rc, ra):
    qc = QuantumCircuit(rc, ra)
    qc.cx(ra[1],rc[1])
    qc.cx(ra[0],rc[2])
    qc.cx(ra[2],rc[2])
    return qc

# 양자 레지스터와 고전 레지스터를 정의하고 이들을 사용하는 양자 회로를 초기화
n = 3
ra = QuantumRegister(n,'a')
rc = QuantumRegister(n,'c')
qc = QuantumCircuit(rc, ra)

# n=3 인 사이먼 오라클 회로를 양자 회로로 구현
oracle = simon_oracle3(rc, ra)
oracle.draw(output='mpl')


In [ ]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
# 사이먼 회로의 구현
n = 3
ra = QuantumRegister(n,'a')
rc = QuantumRegister(n,'c')
rma = ClassicalRegister(n,'ma')
rmc = ClassicalRegister(n,'mc')
qc = QuantumCircuit(rc, ra, rmc, rma)

qc.h(ra)
qc.barrier()
qc.compose(simon_oracle3(rc,ra),range(2*n),inplace=True)
qc.barrier()
qc.measure(rc,rmc)
qc.barrier()
qc.h(ra)
qc.barrier()
qc.measure(ra,rma)
qc.draw(output='mpl',fold=-1)

In [ ]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
# n-큐비트 사이먼 오라클 함수를 구현하는 회로와 random b 를 반환하는 회로
def simon_oracle(rc, ra, n):
    qc = QuantumCircuit(rc, ra)
    np.random.seed()
    k = np.random.randint(1,n+1)
    b1 = np.random.permutation(n)[:k] 
    b = []
    for i in range(n):
        if i in b1: b += [1]
        else :      b += [0] # 임의의 non-zero b 를 생성

    qc.barrier()
    for i in range(n): # c[i] ^= a[i]
        qc.cx(ra[i],rc[i])
    qc.barrier() 
    for bi in b1:      # c^f(x^b) 를 c^f(x) 와 같게 만듦
        qc.cx(ra[n-1-b1[0]],rc[n-1-bi])

    return qc,b

# 양자 레지스터와 고전 레지스터를 정의하고 이들을 사용하는 양자 회로를 초기화
n = 5
ra = QuantumRegister(n,'a')
rc = QuantumRegister(n,'c')
rm = ClassicalRegister(n,'m')
qc = QuantumCircuit(rc, ra, rm)

# 사이먼 오라클 회로를 양자 회로로 구현
oracle, b = simon_oracle(rc,ra,n)
print(f"b={b}")
oracle.draw(output='mpl', fold=-1)


In [ ]:
# 사이먼 회로의 구현
n = 5
ra = QuantumRegister(n,'a')
rc = QuantumRegister(n,'c')
rma = ClassicalRegister(n,'ma')
rmc = ClassicalRegister(n,'mc')
qc = QuantumCircuit(rc, ra, rmc, rma)

qc.h(ra)
qc.barrier()
oracle, b = simon_oracle(rc,ra,n)
qc.compose(oracle,range(2*n),inplace=True)
qc.barrier()
qc.measure(rc,rmc)
qc.barrier()
qc.h(ra)
qc.barrier()
qc.measure(ra,rma)
qc.draw(output='mpl',fold=-1)

In [ ]:
# AerSimulator 를 이용하여 circuit 회로를 측정한 결과를 출력
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import SamplerV2 as Sampler
from qiskit_aer import AerSimulator
aer_sim = AerSimulator()
pm = generate_preset_pass_manager(backend=aer_sim, optimization_level=1)
isa_circuit = pm.run(qc)
sampler = Sampler(mode=aer_sim)
job = sampler.run([isa_circuit], shots=n+20)
result = job.result()
count = result[0].data.ma.get_counts()
print(count)


In [ ]:
import numpy as np                                      #type: ignore
def rank_mod2(matrix):
    rows, cols = matrix.shape
    rank = 0
    for c in range(cols):
        # find the pivot row in column c
        for r in range(rank, rows):
            if matrix[r,c] != 0:
                # swap rows if needed
                if r != rank:
                    matrix[[rank,r]] = matrix[[r,rank]]

                # eliminate column below 
                for k in range(rank+1, rows):
                    if matrix[k,c] == 1:
                        matrix[k] ^= matrix[rank] 
                rank += 1
                break
    return rank

def indep(sys,e):
    sys.append(e)
    matrix = np.array(sys)
    n = len(sys)
    if rank_mod2(matrix) == n:
        return True
    else:  
        sys.pop()
        return False

#  Gaussian Elimination in modulo-2
def ge_mod2(A):
    n = len(A)+1
    matrix = np.matrix(A)
    # Gaussian elimination
    for i in range(n-1):
        # Find the pivot row
        pivot = i
        while pivot < (n-1) and matrix[pivot,i] == 0:
            pivot += 1            
        if pivot < (n-1):
            matrix[[i,pivot]] = matrix[[pivot,i]] # swap rows
            # Eliminate rows below 
            for j in range(i + 1, n-1):
                if matrix[j, i] == 1:
                    matrix[j] ^= matrix[i]  # Row operation with XOR
    return matrix

#  Back Substitution in modulo-2
def back_substitution_mod2(matrix):
    # Back substitution to get a NON TRIVIAL solution
    _, n = matrix.shape
    x = -2*np.ones(n, dtype=int) # x[i] == -1 means x[i] is unknown yet    
    x[-2] = x[-1] = -1                    # last row : 0....0011
    if matrix[n-2,-1] == 0 : x[-2] = 0    # last row : 0....0010
    elif matrix[n-2,-2] == 0 : x[-1] = 0  # last row : 0....0001
    for r in range(n-3, -1, -1):
        if matrix[:,r].sum() == 0 : 
            x[r] = 1  
        value = 0
        for c in range(r,n):
            if matrix[r,c] == 1 :
                if x[c] < 0 : break
        unknown = 0
        for j in range(c + 1, n):
            if matrix[r,j] == 1:
                if x[j] == -2: x[j] = -1
                if x[j] == -1:  unknown = 1 - unknown
                else :          value ^=  x[j]  # XOR to solve mod-2 equations
        if unknown == 1 : x[c] = -1
        else :            
            if x[c] == -1:
                for j in range(r+1, n):
                    if x[j] == -1: x[j] = value
            x[c] = value
    # determine the unknown
    x1 = x.copy()
    for i in range(n):
        if x[i] == -1 : x1[i] = 1
    for r in range(n-1):
        v = 0
        for c in range(n):
            v ^= matrix[r,c] & x1[c]
        if v == 1 :
            for i in range(n):
                if x[i] == -1 : x[i] = 0
            return x
    return x1


def solve_simon(measured):
    sys_eq = []
    for z in measured.keys():
        n = len(z)
        if int(z) == 0 : continue  # skip all 0's
        eq = [int(zi) for zi in z] # binary string -> int list        
        if len(sys_eq) == 0:
            sys_eq.append(eq)
            continue  
        if (indep(sys_eq, eq)):
            if (len(sys_eq) == (n-1)):
                break
    print("The following z's are measured : ")
    print(np.array(sys_eq))
    matrix = ge_mod2(sys_eq)
    print("REF matrix:")
    print(matrix)
    b = back_substitution_mod2(matrix)
    return b

print("measured output : ", count)
b_simon = solve_simon(count)
print(f"The period of a function f(x)) with Simon Property is {b_simon}")